# Dirty Data Cleaning with Autoencoder & Genetic Algorithm
Projekt do czyszczenia danych przy użyciu AI: autoenkodera (deep learning) oraz algorytmu genetycznego (optymalizacja heurystyczna).

In [ ]:
# Importy
import pandas as pd
import numpy as np

In [ ]:
# Wczytanie danych
df = pd.read_csv('../data/dirty_data.csv')
df.head()

##  Przygotowanie danych (konwersja, imputacja braków itd.)

In [ ]:
# Przykład czyszczenia - uzupełnianie wieku poza zakresem i NaN
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['age'] = df['age'].mask((df['age'] < 0) | (df['age'] > 120))
df['age'] = df['age'].fillna(df['age'].median())
df['income'] = pd.to_numeric(df['income'], errors='coerce')
df['income'] = df['income'].mask((df['income'] < 1000) | (df['income'] > 100000))
df['income'] = df['income'].fillna(df['income'].median())
df.head()

## Budowa prostego autoenkodera (MLP, TensorFlow)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler

# Zakładamy, że dane są numeryczne
X = df[['age', 'income']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Autoencoder
inp = Input(shape=(2,))
enc = Dense(4, activation='relu')(inp)
enc = Dense(2, activation='relu')(enc)
dec = Dense(4, activation='relu')(enc)
out = Dense(2, activation='linear')(dec)
model = Model(inp, out)
model.compile(optimizer='adam', loss='mse')
model.fit(X_scaled, X_scaled, epochs=20, verbose=1)

In [ ]:
# Detekcja błędów (rekonstrukcja)
reconstructed = model.predict(X_scaled)
mse = np.mean(np.power(X_scaled - reconstructed, 2), axis=1)
df['anomaly_score'] = mse
df.sort_values(by='anomaly_score', ascending=False).head()

##  Podsumowanie
- Wykryto anomalie na podstawie rekonstrukcji autoenkodera
- Możesz teraz dodać algorytm genetyczny w celu optymalizacji czyszczenia
- Rozszerzenie: obsługa tekstu (embedding), logika reguł, integracja GUI